# Variables to update can be found in checking_domains_setup.py

In [1]:
from checking_domains_setup import *

os.chdir(base_dir)

# Code to visualise phylogenetic tree

In [2]:
from ete3 import Tree, TreeStyle, TextFace, add_face_to_node, SeqMotifFace, NodeStyle, faces, ImgFace, CircleFace, AttrFace
from IPython.display import display

def get_domains(domains):
    
    pos_dict = {'RBD_A' : 0, 'RBD_C' : 1, 'RBD_B' : 2, 'NMD' : 3, 'RBD_D' : 4, 'TcB_BD_seed' : 5}

    
    domain_list = [
        # seq.start, seq.end, shape, width, height, fgcolor, bgcolor
        [10,  70, "[]", None, 20, "black", "rgradient:lightgreen", "arial|3|black|RBD_A"],
        [80, 140, "[]", None, 20, "black", "rgradient:blue", "arial|3|black|RBD_C"],
        [150, 210, "[]", None, 20, "black", "rgradient:orange", "arial|3|black|RBD_B"],
        [220, 280, "[]", None, 20, "black", "rgradient:purple", "arial|3|black|NMD"],
        [290, 350, "[]", None, 20, "black", "rgradient:gray", "arial|3|black|RBD_D"],
        [360, 430, "[]", None, 20, "black", "rgradient:darkgreen", "arial|3|black|TCB_BD"]]
    
    
    for k,v in pos_dict.items():
        if k not in domains:
            domain_list[v][6] = 'white'
            domain_list[v][7] = "arial|3|black|"
            
    return domain_list


def get_example_tree(tree, tag_dict, colour_dict, region_dict):
            
            colour = None

            # Label all internal nodes
            edge = 0
            for node in tree.traverse():
                if not node.is_leaf():
                    node.name = "N%d" % edge
                    edge += 1

            # Get the colours for each extant genome
            for node in tree.iter_descendants("postorder"):
                if node.is_leaf():
                    long_name = node.name.split("_joined")[0]
                    short_name = node.name.split("_information")[0]
                    
                    if long_name in tag_dict:
                        colour = colour_dict[tag_dict[long_name][0]]
                    elif short_name in tag_dict:
                        colour = colour_dict[tag_dict[short_name][0]]
                    else:
                        colour = 'black'
                    spaced_name = " ".join(node.name.split("_")[3:5])

                    nameFace = TextFace("  " + spaced_name, fsize=15, fgcolor='black')
                    
                    box_domains = get_domains([x for x in region_dict[node.name].keys()])
                    
                    seqFace = SeqMotifFace(seq=None, motifs=box_domains, gap_format="line")
                    node.add_face(seqFace, 0, "aligned")

                    node.add_face(nameFace, column=0)

                else:
                    colour = 'black'
                    
                if colour == None:
                    colour = 'black'

                nstyle = NodeStyle()
                nstyle["fgcolor"] = colour
                nstyle["size"] = 20
                node.set_style(nstyle)
                
            
            
                ts = TreeStyle()
                ts.show_leaf_name = False
                ts.branch_vertical_margin = 10

                # if custom_layout:
                #     ts.layout_fn = layout
                #     ts.show_leaf_name = False

            # ts.mode = "c"
            ts.root_opening_factor = 1

            return tree, ts

def colour_tips(tree, tag_dict, colour_dict, region_dict, outpath=None, custom_layout=False):

    tree, ts = get_example_tree(tree, tag_dict, colour_dict, region_dict)
    if outpath:
        tree.render(outpath, dpi=300, tree_style=ts)

    display(tree.render('%%inline', dpi=300, tree_style=ts))
    return tree, ts

# Code to run the alignment, tree inference, domain searching

In [3]:
def load_profile_dict(base_dir, profile_name):
    
    print (f'Profile name is {profile_name}\n')

    profile_dir = f'{base_dir}/profiles/{profile_name}'
    
    print (f'Profile dir is {profile_dir}\n')
    profile_file = glob.glob(f'{profile_dir}/*.p')[0]

    # Load the profile dict
    with open(profile_file, 'rb') as file_path:
        profile_dict = pickle.load(file_path)
    
    return profile_dict


def setup_working_dir(base_dir, filename):
    # Creating a working directory
    if not os.path.isdir(f'{base_dir}/results/{filename}'):
        print ("Making directory\n")
            
        # Make the directory
        os.mkdir(f'{base_dir}/results/{filename}')
        
        # Copy the FASTA file into the working directory
        os.popen(f'cp {filename}.fasta ./results/{filename}/{filename}.fasta ')
        
        while not os.path.isfile(f'./results/{filename}/{filename}.fasta'):
            os.wait()

    file_dir = f'{base_dir}/results/{filename}/'

    return file_dir

def make_alignment(filename):
    
    # Make an alignment
    if not os.path.isfile(filename + '.aln'):
        print ("Creating alignment\n")
        while not os.path.isfile(f'{filename}.fasta'):
            os.wait()
        stdoutdata = subprocess.getoutput(f'mafft --genafpair --maxiterate 1000 --reorder {file_dir}{filename}.fasta > {filename}.aln')

        print ("Alignment created\n")
        
def make_tree(filename):
    
    # Make a tree
    if not os.path.isfile(f'RAXML_bestTree.{filename}.nwk'):
        print ("Creating tree\n")

        stdoutdata = subprocess.getoutput(f'raxml -m PROTGAMMAJTT -p 23456 -n {filename}.nwk -s {filename}.aln')

        print ("Tree created\n")

def place_outgroup(filename, outgroup):
    # Place the outgroup

    if not os.path.isfile(f'{filename}.nwk'):
        print ("Placing outgroup\n")

        tree = Tree(f'RAXML_bestTree.{filename}.nwk', format=1)
        tree.set_outgroup(outgroup)

        tree.write(outfile= f'{filename}.nwk', format=1)

def search_for_domains(base_dir, file_dir, profile_name):
    print ("Searching for domains \n")
    input_folder, output_folder, profile_folder, tree_folder = setup_domain_directories(base_dir, file_dir, profile_name)
    region_dict = search_seqs_with_profiles(input_folder, output_folder, profile_folder, tree_folder)
    region_dict, domain_dict = process_hmmer_results(region_dict, output_folder)
    
#     for path in [output_folder + '/*.output', input_folder + 'tmp/*' ]:
#         remove_temp_files(path)
    
    return region_dict, domain_dict

    
def setup_domain_directories(base_dir, file_dir, profile_name):
    for dir_name in ['domain_input', 'domain_output', 'tree_output']:
        if not os.path.isdir(dir_name):
            os.mkdir(dir_name)
    
    print ('Setup the domain directories \n')
    
    input_folder = f'{file_dir}domain_input/'
    output_folder = f'{file_dir}domain_output/'
    profile_folder = f'{file_dir}{profile_name}/'
    tree_folder = f'{file_dir}tree_output/'
    
    

    shutil.copy2(f'{file_dir}{filename}.fasta', f'{input_folder}{filename}.fasta')
    if not os.path.isdir(f'{profile_folder}'):
        shutil.copytree(f'{base_dir}/profiles/{profile_name}', f'{profile_folder}')
    

    while not os.path.isdir(f'{file_dir}{profile_name}'):
        os.wait()
    
    return input_folder, output_folder, profile_folder, tree_folder

def search_seqs_with_profiles(input_folder, output_folder, profile_folder, tree_folder):
    
    print ('Searching the sequences with the profiles\n')
    domScore = 1

    region_dict = {}

    files = [x for x in os.listdir(input_folder) if x != ".DS_Store" and x != 'tmp']
    profiles = glob.glob(profile_folder + "/*.hmm")
    
    for file in files:

        seqs = sequence.readFastaFile(input_folder + file)

        # Write out each region in the file to a separate file

        if not os.path.isdir(input_folder + "tmp"):
            os.mkdir(input_folder + "tmp")

        for seq in seqs:
            sequence.writeFastaFile(input_folder + "tmp/" + seq.name, [seq])

            if seq.name not in region_dict:
                region_dict[seq.name] = {}

        # Check each region with each profile
            for profile in profiles:
                profile_split =  profile.split("/")[-1]
                stdoutdata = subprocess.getoutput(f'hmmsearch -o {output_folder}{seq.name}_profile={profile_split}.output --domT {domScore} {profile_name}/{profile_split} {input_folder}tmp/{seq.name}')
        
    return region_dict

def process_hmmer_results(region_dict, output_folder):
    domain_dict = {}
    
    for infile in glob.glob(output_folder + '/*.output'):

        seq_name = infile.split(output_folder)[1].split("_profile=")[0]

        qresult = SearchIO.read(infile, 'hmmer3-text')
        if len(qresult.hits) > 0:
            hsp = qresult[0][0]
            domain = infile.split("profile=")[1].split(".")[0]
            start = hsp.hit_start
            end = hsp.hit_end
            region_dict[seq_name][domain] = (start, end)

            if domain not in domain_dict:
                domain_dict[domain] = []
            domain_dict[domain].append(seq_name)
    return region_dict, domain_dict



def check_for_domains_to_add(profile_dict, domain_dict):

    domains_to_add = defaultdict(list)
    for domain, seq_list in profile_dict.items():
        if not seq_list[0] == 'Pfam_seed':
            if domain in domain_dict.keys():
                for found_domain in domain_dict[domain]:
                    if found_domain not in profile_dict[domain]:
                        domains_to_add[domain].append(found_domain)
    
    return domains_to_add

def add_domains(domains_to_add, region_dict, profile_dict, profile_name):
        
    for domain_name, to_add in domains_to_add.items():
        
            print (domain_name)
            print (to_add)
            seqs_to_add = []
            

            original_seqs = sequence.readFastaFile(f'{profile_name}/{domain_name}.fasta')

            add_short = [" ".join(x.split("information_")[1].split("_")[0:2]) for x in to_add]

        
            if len(add_short) > 1:
                seq_txt = 'sequences'
            else:
                seq_txt = 'sequence'
    

            print (f'For domain {domain_name} we have to add {len(add_short)} {seq_txt}\n')
            print (f'Sequences where we found domains are - {", ".join(add_short)}\n')
            for add in to_add:
                   
                
                   start = region_dict[add][domain_name][0]
                   end = region_dict[add][domain_name][1]
                   
                   # Add them to the profile dict
                   profile_dict[domain_name].append(add)

                   for seq in regions:
                       if seq.name == add:
                           seq_to_add = sequence.Sequence("".join(seq.sequence[start:end]), name=f'{add}_{domain_name}')
                           seqs_to_add.append(seq_to_add)


            sequence.writeFastaFile(f'{profile_name}/{domain_name}.fasta', original_seqs + seqs_to_add)


            # Delete the old alignment and HMM
            os.remove(f'{profile_name}/{domain_name}.aln')
            os.remove(f'{profile_name}/{domain_name}.hmm')

            # Make new alignment
            stdoutdata = subprocess.getoutput(f'mafft --genafpair --maxiterate 1000 --reorder {profile_name}/{domain_name}.fasta > {profile_name}/{domain_name}.aln')

            # Make new HMM
            stdoutdata = subprocess.getoutput(f'hmmbuild {profile_name}/{domain_name}.hmm {profile_name}/{domain_name}.aln')
    return profile_dict

                   
def search_domains(base_dir, filename, file_dir, profile_name, count = 0):
                   
                   
    # Load the tree        
    tree = Tree(f'{filename}.nwk', format=1)

    region_dict, domain_dict = search_for_domains(base_dir, file_dir, profile_name)
    profile_dict = load_profile_dict(base_dir, profile_name)

    domains_to_add = check_for_domains_to_add(profile_dict, domain_dict)
                   
    print ('This is the current tree \n')
                   
    output_tree = display_tree(tree, region_dict, profile_name)


    if domains_to_add:
        profile_dict = add_domains(domains_to_add, region_dict, profile_dict, profile_name)
        
        count +=1
        new_profile_dict_name = f'{profile_name}_{filename}_{count}'
        save_new_profile_dict(profile_dict, profile_name, new_profile_dict_name)
                   
        if not os.path.isdir(f'{base_dir}/profiles/{new_profile_dict_name}'):

                   
            shutil.copytree(f'{profile_name}', f'{base_dir}/profiles/{new_profile_dict_name}')

        search_domains(base_dir, filename, file_dir, f'{profile_name}_{filename}_{count}', count)

    else:
        print ('No new domains discovered')


                   

def save_new_profile_dict(profile_dict, profile_name, new_profile_dict_name):
    remove_temp_files(profile_name + "/*.p")
                   

    with open(f'{profile_name}/{new_profile_dict_name}.p', 'wb') as profile_path:
                   pickle.dump(profile_dict, profile_path)
                   


def remove_temp_files(path):
    files = glob.glob(path)
    for f in files:
        os.remove(f)
                   
def display_tree(tree, region_dict, filename):

    # Save the coloured tips to file
    output_tree, ts = colour_tips(tree, tag_dict, colour_dict, region_dict, outpath= f'tree_output/{filename}.png', custom_layout=False)
                                      
    return (output_tree, ts)

# Main method for running the workflow

In [ ]:
import sequence
import shutil
import subprocess
import shutil
import glob
import pickle
from collections import defaultdict
from Bio import SearchIO
from ete3 import Tree, TreeStyle, TextFace, add_face_to_node, SeqMotifFace, NodeStyle, faces, ImgFace, CircleFace, AttrFace

# Create and setup the working directory
file_dir = setup_working_dir(base_dir, filename)

# Move into the working directory
os.chdir(file_dir)

# Load the sequences
regions = sequence.readFastaFile(f'{filename}.fasta')

# If we haven't already, make an alignment a tree and place the outgroup correctly

make_alignment(filename)
make_tree(filename)
place_outgroup(filename, outgroup)

# Search for domains in the sequences
output_tree = search_domains(base_dir, filename, file_dir, profile_name)

Making directory

Creating alignment

